# Delegated Smart signatures
#### 06.6 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-03-14

* Combine the logic of Smart Signatures with existing accounts

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction, LogicSigAccount

import algosdk.error
import json
import base64
import hashlib

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [5]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VL5UU2QXNKNEH7VISHZFU2ALXN5MOIDD3KXEYX2ADLCCIYN3MCOKRBATV4
VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


#### Check Purestake API

In [6]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 20482125


## Two types of Smart Signatures
* So far, Smart Signatures were used in *account mode* as **contract account**
    * Compiling produces the public address
    * Smart Signature is an independent account
    * Transactions can only be authorized by Smart Signature

#### This has some drawbacks
* We have to forsee all possible cases when defining the logic of the Smart Signature
* We have to cover transactions that happen rarely (withdrawal of funds) or only once (opt-in)

## Better: Delegated Smart Signatures
* The second mode is *delegated mode* as **delegated signature**
* Alice creates a Smart Signature and signs it as delegeted signature
* Transactions with Alice's account can be authorized ...
    * With Alice's privat key
    * With the Smart signature

## Create a Delegated Signature
* Alice wants to create a delegated Smart Signature that Bob can retrieve 0.1 ALGO from her account
* She wants to be able to authorize all other transactions herself

**Note** Steps 1-3 are identacal in account mode and delegated mode 

#### Step 1: Define conditions in PyTEAL

In [7]:
# withdraw condition
withdraw_cond = And (
    Txn.receiver() == Addr(Bob["public"]),        # Receipient must be Bob
    Txn.amount() == Int(int(1e5)),                # 0.1 ALGO = 1e5 Micro Algo
)

# random condition
import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = ( Txn.fee() <= Int(1000) )

safety_cond = And(
    Txn.type_enum() == TxnType.Payment,
    Txn.close_remainder_to() == Global.zero_address(),
    Txn.rekey_to() == Global.zero_address(),
    )
    
withdraw_pyteal = And(
    withdraw_cond,
    random_cond, 
    fee_cond, 
    safety_cond
    )

##### Step 2: Compile PyTeal -> Teal

In [8]:
withdraw_teal = compileTeal(withdraw_pyteal, Mode.Signature, version=3)
#print(bob_teal)

#### Step 3: Compile Teal -> Bytecode for AVM

In [9]:
Withdraw = algod_client.compile(withdraw_teal)
encodedProg = Withdraw['result'].encode()
program = base64.decodebytes(encodedProg)
# lsig = LogicSig(program)                   # old version, use LogicSigAccount instead
lsigA = LogicSigAccount(program)

#### Step 4: Alice signs the Smart Signature
* Additional step to link conditions (step 1) to the account of Alice

In [10]:
# lsig.sign(Alice['private'])               # old version, use lsigA.sign() instead
lsigA.sign(Alice['private'])

#### The Signed Smart Signature is ready for use
#### Alice communicates the following pieces of information to Bob

In [11]:
# (1) The program logic (the rules)
withdrawal_prog = encodedProg
withdrawal_prog

b'AyAEoI0GwISa5wvoBwEmASAT6T3nokbu5GeAjI1CwH+vMGwm3QlBll79w6vFJEpHmDEHKBIxCCISECMjEhAxASQOEDEQJRIxCTIDEhAxIDIDEhAQQw=='

In [12]:
# (2) The signature of the program
withdrawal_sig = lsigA.lsig.sig
withdrawal_sig

'ir3TvgpYIVv7V0uCcidTrmNkEhKmk5e3gEBXL+yDTPL5Xy+l9Z6fN0/h6NJE7x6nHHw7oMT+uUqwVpLv8q0GDA=='

In [13]:
# (3) The signing account (=Alice['public'])
withdrawal_acc = base64.b32encode(lsigA.sigkey)
withdrawal_acc

b'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBNQ===='

#### Bob recreates the smart signature from these pieces of information

In [15]:
# Step 1: create a new smart signature object from the program logic
lsigBob = LogicSigAccount(base64.decodebytes(withdrawal_prog))
# Step 2: add the signature (i.e. that Alice has signed the logic)
lsigBob.lsig.sig = withdrawal_sig
# Step 3: add the public key of the siging account (Alice)
lsigBob.sigkey = base64.b32decode(withdrawal_acc)

In [16]:
lsigBob.address()

'VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU'

## Two possible transactions from the Alice account

#### Case A: Transaction signed by Smart Signature
* Note that we do not have to "fund the Smart Signature"
* The Smart Signature authorizes trasactions from Alice's account

In [160]:
# Step 1: prepare
sp = algod_client.suggested_params()
amt = int(1e5)
sender = Alice['public']             # <--------- NEW: Sender is Alice (not the Smart Sig)
receiver = Bob['public']
txn = PaymentTxn(sender, sp, receiver, amt)

# Step 2: Smart Sig signs txn
lstx = transaction.LogicSigTransaction(txn, lsigBob)   # <--------- Signature recreated by Bob 

# Step 3: Send
txid = algod_client.send_transaction(lstx)

# Step 4: Wait for confirmation
confirmed_txn = wait_for_confirmation(algod_client, txid)

Current round is  20346987.
Waiting for round 20346987 to finish.
Waiting for round 20346988 to finish.
Transaction IXS5WXJJHAC2G527ATE5IVGJ4BFBZH2SDI6O77XVDLE6GGKO6HSA confirmed in round 20346989.


#### Case B: Transaction signed by Alice
* Alice can still sign for herself
* She can of cause sign all sorts of transactions, not only the ones in the Smart Signature
    * Example: she sends 0.2 ALGO to Charlie

In [69]:
# Step 1: prepare
sp = algod_client.suggested_params()
amt = int(2e5)
sender = Alice['public']
receiver = Charlie['public']
txn = PaymentTxn(sender, sp, receiver, amt)

# Step 2+3+4: sign, send, wait
stxn = txn.sign(Alice['private'])                    # <------ nothing new here. Alices signs with private key           
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20345424.
Waiting for round 20345424 to finish.
Waiting for round 20345425 to finish.
Transaction 2QPJMKVEUDBLJDDZ3A7FSAZ2VE4UHJUOPAC6DTCKTPC7G2C7QCNQ confirmed in round 20345426.


## Possible applications
* Let the Smart Signature handle standard transactions
* Manually sign exceptional or rare transactions (like opt-in, withdrawal of funds from vending, ...)